In [33]:
import numpy as np
from serpentGenerator.functions.pin import pin
from serpentGenerator.data.materialLibrary import MATLIB
from serpentGenerator.functions.material import material
from serpentGenerator.functions.sqLattice import sqLat
from serpentGenerator.functions.hexLattice import hexLat
from serpentGenerator.functions.pinStack import pinStack
from serpentGenerator.functions.surfs import surfs
from serpentGenerator.functions.surf import surf
from serpentGenerator.functions.cell import cell
from serpentGenerator.functions.cells import cells
from serpentGenerator.functions.lats import lats
from serpentGenerator.functions.mats import mats
from serpentGenerator.functions.pins import pins
from serpentGenerator.functions.core import core
from serpentGenerator.functions.housing import housing
from serpentGenerator.functions.universe import universe

In [34]:
fuel1 = MATLIB['UO2'].duplicateMat("fuel1")
fuel1.set('fractions', np.array([-0.04319337211, -0.83830401789, -.11850261])) # 4.9 % wt enrichment
fuel1.set('rgb', "238 59 49")

fuel2 = fuel1.duplicateMat("fuel2")
fuel2.set('rgb', "255 165 0")

fuel3 = fuel1.duplicateMat("fuel3")
fuel3.set('rgb', "255 255 0")

mod = MATLIB['lightWater'].duplicateMat("water")
mod.set("isBurn", False)
mod.set('rgb', "198 226 255")

clad = MATLIB['Zr'].duplicateMat("clad")
clad.set("isBurn", False)
clad.set('rgb', "183 183 183")

ref = MATLIB['plenum'].duplicateMat("reflector") # reflector plenum
ref.set("isBurn", False)
ref.set('rgb', "204 153 255")

ba = MATLIB['boron'].duplicateMat("B4C")
ba.set("isBurn", False)
ba.set('rgb', "255 255 0")

matList = mats()
matList.addMats([fuel1, fuel2, fuel3,  mod, clad, ref, ba])

print(matList.toString())

mat fuel1    -10.2141  burn 1  rgb 238 59 49
92235.09c	-0.04319337211
92238.09c	-0.83830401789
8016.09c	-0.11850261

mat fuel2    -10.2141  burn 1  rgb 255 165 0
92235.09c	-0.04319337211
92238.09c	-0.83830401789
8016.09c	-0.11850261

mat fuel3    -10.2141  burn 1  rgb 255 255 0
92235.09c	-0.04319337211
92238.09c	-0.83830401789
8016.09c	-0.11850261

mat water    -0.7 moder ltwr 1001   rgb 198 226 255
1001.06c	0.66667
8016.06c	0.33333

mat clad    -6.55    rgb 183 183 183
40090.06c	0.5145
40091.06c	0.1122
40092.06c	0.1715
40094.06c	0.1738
40096.06c	0.028

mat reflector    0.070462    rgb 204 153 255
40090.06c	0.0059311
40091.06c	0.0012934
40092.06c	0.001977
40094.06c	0.0020035
40096.06c	0.00032278
26054.06c	0.011217
26056.06c	0.17738
26057.06c	0.0042545
26058.06c	0.00054149
24050.06c	0.0024673
24052.06c	0.04758
24053.06c	0.0053951
24054.06c	0.001343
28058.06c	0.017122
28060.06c	0.0065978
28061.06c	0.00028421
28062.06c	0.00091398
28064.06c	0.0002329
25055.06c	0.0056687
1001.06c	0.47165
80

In [35]:
F1 = pin("F1", 3)
F1.set('materials', np.array([fuel1, mod, clad]))
F1.set('radii', np.array([.4095, 4750]))

F2 = F1.duplicatePin("F2")
F2.set('materials', np.array([fuel2, mod, clad]))

F3 = F1.duplicatePin("F3")
F3.set('materials', np.array([fuel3, mod, clad]))

# G1 = pin("G", 3)
# G1.set('materials', np.array([mod, clad, mod]))
# G1.set('radii', np.array([.5715, 6120]))

W = pin("W", 3)
W.set('materials', np.array([mod]))

RT = pin("RT", 1) #top reflector
RT.set('materials', np.array([ref]))

RB =  RT.duplicatePin("RB")

pinList = pins()
pinList.addPins([F1, F2, F3, W, RT, RB])

#print(pinList.toString())

In [36]:
riClad = .57150
roClad = .61200

tClad = roClad - riClad

span1 = 70
gap1 = (360-span1*4)/4
theto1 = (90 - span1)/2
thetf1 = 360 - theto1

span2 = 55
gap2 = (360-span2*4)/4
theto2 = (90 - span2)/2
thetf2 = 360 - theto2

span3 = 60
gap3 = (360-span3*4)/4
theto3 = (90 - span3)/2
thetf3 = 360 - theto3

t1 = .090/10
t2 = .089/10
t3 = .019/10

riPad1 = riClad+ (tClad - t1)/2 
roPad1 = riPad1 + t1

riPad2 = riClad+ (tClad - t2)/2 
roPad2 = riPad2 + t2

riPad3 = riClad+ (tClad - t3)/2 
roPad3 = riPad3 + t3

r1 = [riClad, riPad1, roPad1, roClad]
r2 = [riClad, riPad2, roPad2, roClad]
r3 = [riClad, riPad3, roPad3, roClad]

crs1 = [None]*len(r1)
baps1 = [None]*len(r1)
bapscc1 = [None]*len(r1)
clps1 = [None]*len(r1)
clpscc1 = [None]*len(r1)

crs2= [None]*len(r2)
baps2= [None]*len(r2)
bapscc2 = [None]*len(r2)
clps2= [None]*len(r2)
clpscc2= [None]*len(r2)

crs3= [None]*len(r3)
baps3= [None]*len(r3)
bapscc3 = [None]*len(r3)
clps3= [None]*len(r3)
clpscc3= [None]*len(r3)


alpha11= theto1
alpha12 = alpha11 + span1
alpha13 = thetf1
alpha14 = (alpha13 + gap1) % 360
                    
alpha21 = theto2
alpha22 = alpha21 + span2
alpha23 = thetf2
alpha24 = (alpha23 + gap2) % 360

alpha31 = theto3
alpha32 = alpha31 + span3
alpha33 = thetf3
alpha34 = (alpha33 + gap3) % 360

for i in range(0, len(r1)):
    crs1[i] = surf("crt1"+str(i+1), "cyl", np.array([0.0, 0.0, r1[i]]))
    baps1[i] = surf("bat1"+str(i+1), "pad", np.array([0.0, 0.0, riPad1, roPad1, alpha11, alpha12]))
    clps1[i] = surf("clt1"+str(i+1), "pad", np.array([0.0, 0.0, riPad1, roPad1, alpha13, alpha14]))
    bapscc1[i] = cell("bapadt1"+str(i+1), np.array([baps1[i]]), np.array([1]), ba.id)
    clpscc1[i] = cell("clpadt1"+str(i+1), np.array([clps1[i]]), np.array([1]), clad.id)
    alpha11 = alpha12 + gap1
    alpha12 = alpha11 + span1
    alpha13 = alpha14 + span1
    alpha14 = alpha13 + gap1
    
for i in range(0, len(r2)):
    crs2[i] = surf("crt2"+str(i+1), "cyl", np.array([0.0, 0.0, r2[i]]))
    baps2[i] = surf("bat2"+str(i+1), "pad", np.array([0.0, 0.0, riPad2, roPad2, alpha21, alpha22]))
    clps2[i] = surf("clt2"+str(i+1), "pad", np.array([0.0, 0.0, riPad2, roPad2, alpha23, alpha24]))
    bapscc2[i] = cell("bapadt2"+str(i+1), np.array([baps2[i]]), np.array([1]), ba.id)
    clpscc2[i] = cell("clpadt2"+str(i+1), np.array([clps2[i]]), np.array([1]), clad.id)
    alpha21 = alpha22 + gap2
    alpha22 = alpha21 + span2
    alpha23 = alpha24 + span2
    alpha24 = alpha23 + gap2
    
for i in range(0, len(r3)):
    crs3[i] = surf("crt3"+str(i+1), "cyl", np.array([0.0, 0.0, r3[i]]))
    baps3[i] = surf("bat3"+str(i+1), "pad", np.array([0.0, 0.0, riPad3, roPad3, alpha31, alpha32]))
    clps3[i] = surf("clt3"+str(i+1), "pad", np.array([0.0, 0.0, riPad3, roPad3, alpha33, alpha34]))
    bapscc3[i] = cell("bapadt3"+str(i+1), np.array([baps3[i]]), np.array([1]), ba.id)
    clpscc3[i] = cell("clpadt3"+str(i+1), np.array([clps3[i]]), np.array([1]), clad.id)
    alpha31 = alpha32 + gap3
    alpha32 = alpha31 + span3
    alpha33 = alpha34 + span3
    alpha34 = alpha33 + gap3
    
cc11 = cell("cc11", np.array([crs1[0]]), np.array([1]), mod.id)
cc12 = cell("cc12", np.array([crs1[0], crs1[1]]), np.array([0, 1]), clad.id)
cc13 = cell("cc13", np.array([crs1[2], crs1[3]]), np.array([0, 1]), clad.id)
cc14 = cell("cc14", np.array([crs1[3]]), np.array([0]), mod.id)

cc21 = cell("cc21", np.array([crs2[0]]), np.array([1]), mod.id)
cc22 = cell("cc22", np.array([crs2[0], crs2[1]]), np.array([0, 1]), clad.id)
cc23 = cell("cc23", np.array([crs2[2], crs2[3]]), np.array([0, 1]), clad.id)
cc24 = cell("cc24", np.array([crs2[3]]), np.array([0]), mod.id)

cc31 = cell("cc31", np.array([crs3[0]]), np.array([1]), mod.id)
cc32 = cell("cc32", np.array([crs3[0], crs3[1]]), np.array([0, 1]), clad.id)
cc33 = cell("cc33", np.array([crs3[2], crs3[3]]), np.array([0, 1]), clad.id)
cc34 = cell("cc34", np.array([crs3[3]]), np.array([0]), mod.id)

surfList1 = surfs()
surfList1.addSurfs(crs1)
surfList1.addSurfs(baps1)
surfList1.addSurfs(clps1)

cellList1 = cells()
cellList1.addCells(bapscc1)
cellList1.addCells(clpscc1)
cellList1.addCells([cc11, cc12, cc13, cc14])

bigT1 = universe("bigT1")
bigT1.surfs = surfList1
bigT1.cells = cellList1

print(bigT1.toString())

surfList2 = surfs()
surfList2.addSurfs(crs2)
surfList2.addSurfs(baps2)
surfList2.addSurfs(clps2)

cellList2 = cells()
cellList2.addCells(bapscc2)
cellList2.addCells(clpscc2)
cellList2.addCells([cc21, cc22, cc23, cc24])

bigT2 = universe("bigT2")
bigT2.surfs = surfList2
bigT2.cells = cellList2

# print(bigT2.toString())

surfList3 = surfs()
surfList3.addSurfs(crs3)
surfList3.addSurfs(baps3)
surfList3.addSurfs(clps3)

cellList3 = cells()
cellList3.addCells(bapscc3)
cellList3.addCells(clpscc3)
cellList3.addCells([cc31, cc32, cc33, cc34])

bigT3 = universe("bigT3")
bigT3.surfs = surfList3
bigT3.cells = cellList3

#print(bigT3.toString())

surf crt11 cyl 0.0 0.0 0.5715 
surf crt12 cyl 0.0 0.0 0.58725 
surf crt13 cyl 0.0 0.0 0.5962500000000001 
surf crt14 cyl 0.0 0.0 0.612 
surf bat11 pad 0.0 0.0 0.58725 0.5962500000000001 10.0 80.0 
surf bat12 pad 0.0 0.0 0.58725 0.5962500000000001 100.0 170.0 
surf bat13 pad 0.0 0.0 0.58725 0.5962500000000001 190.0 260.0 
surf bat14 pad 0.0 0.0 0.58725 0.5962500000000001 280.0 350.0 
surf clt11 pad 0.0 0.0 0.58725 0.5962500000000001 350.0 10.0 
surf clt12 pad 0.0 0.0 0.58725 0.5962500000000001 80.0 100.0 
surf clt13 pad 0.0 0.0 0.58725 0.5962500000000001 170.0 190.0 
surf clt14 pad 0.0 0.0 0.58725 0.5962500000000001 260.0 280.0 

cell bapadt11 bigT1  B4C -bat11 
cell bapadt12 bigT1  B4C -bat12 
cell bapadt13 bigT1  B4C -bat13 
cell bapadt14 bigT1  B4C -bat14 
cell clpadt11 bigT1  clad -clt11 
cell clpadt12 bigT1  clad -clt12 
cell clpadt13 bigT1  clad -clt13 
cell clpadt14 bigT1  clad -clt14 
cell cc11 bigT1  water -crt11 
cell cc12 bigT1  clad crt11 -crt12 
cell cc13 bigT1  clad crt13 

In [37]:
FC1 = pinStack("FC1", 0, 0, 3)
FC1.setStack(np.array([RB, F1, RT]), np.array([0.0, 20.0, 220.0]))

FC2 = FC1.duplicateLat("FC2")
FC2.replacePin(F1, F2)

FC3 = FC1.duplicateLat("FC3")
FC3.replacePin(F1, F3)

GT1 = FC1.duplicateLat("GT1")
GT1.replacePin(F1, bigT1)

GT2 = FC1.duplicateLat("GT2")
GT2.replacePin(F1, bigT2)

GT3 = FC1.duplicateLat("GT3")
GT3.replacePin(F1, bigT3)

WG1 = FC1.duplicateLat("WG1")
WG1.replacePin(F1, W)

latList = lats()
latList.addLats([FC1, FC2, FC3, GT1, GT2, GT3, WG1])


print(GT1.toString())

lat GT1 9 0 0 3
0.0	RB
20.0	bigT1
220.0	RT

surf crt11 cyl 0.0 0.0 0.5715 
surf crt12 cyl 0.0 0.0 0.58725 
surf crt13 cyl 0.0 0.0 0.5962500000000001 
surf crt14 cyl 0.0 0.0 0.612 
surf bat11 pad 0.0 0.0 0.58725 0.5962500000000001 10.0 80.0 
surf bat12 pad 0.0 0.0 0.58725 0.5962500000000001 100.0 170.0 
surf bat13 pad 0.0 0.0 0.58725 0.5962500000000001 190.0 260.0 
surf bat14 pad 0.0 0.0 0.58725 0.5962500000000001 280.0 350.0 
surf clt11 pad 0.0 0.0 0.58725 0.5962500000000001 350.0 10.0 
surf clt12 pad 0.0 0.0 0.58725 0.5962500000000001 80.0 100.0 
surf clt13 pad 0.0 0.0 0.58725 0.5962500000000001 170.0 190.0 
surf clt14 pad 0.0 0.0 0.58725 0.5962500000000001 260.0 280.0 

cell bapadt11 bigT1  B4C -bat11 
cell bapadt12 bigT1  B4C -bat12 
cell bapadt13 bigT1  B4C -bat13 
cell bapadt14 bigT1  B4C -bat14 
cell clpadt11 bigT1  clad -clt11 
cell clpadt12 bigT1  clad -clt12 
cell clpadt13 bigT1  clad -clt13 
cell clpadt14 bigT1  clad -clt14 
cell cc11 bigT1  water -crt11 
cell cc12 bigT1  cla

In [38]:
FA1 = sqLat("FA1", 0, 0, 19, 1.260)

FA1Map = np.array([
    [WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, GT1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, GT1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, GT1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, GT1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1]
])

FA1.setMap(FA1Map)

FA2 = FA1.duplicateLat("FA2")
FA2.replacePin(FC1, FC2)
FA2.replacePin(GT1, GT2)

FA3 = FA1.duplicateLat("FA3")
FA3.replacePin(FC1, FC3)
FA3.replacePin(GT1, GT3)

WA1 = FA1.duplicateLat("WA1")
WA1.replacePin(FC1, WG1)
WA1.replacePin(GT1, WG1)



In [39]:
latList.addLats([FA1, FA2, FA3, WA1])

print(latList.toString())

lat FC1 9 0 0 3
0.0	RB
20.0	F1
220.0	RT

lat FC2 9 0 0 3
0.0	RB
20.0	F2
220.0	RT

lat FC3 9 0 0 3
0.0	RB
20.0	F3
220.0	RT

lat GT1 9 0 0 3
0.0	RB
20.0	bigT1
220.0	RT

surf crt11 cyl 0.0 0.0 0.5715 
surf crt12 cyl 0.0 0.0 0.58725 
surf crt13 cyl 0.0 0.0 0.5962500000000001 
surf crt14 cyl 0.0 0.0 0.612 
surf bat11 pad 0.0 0.0 0.58725 0.5962500000000001 10.0 80.0 
surf bat12 pad 0.0 0.0 0.58725 0.5962500000000001 100.0 170.0 
surf bat13 pad 0.0 0.0 0.58725 0.5962500000000001 190.0 260.0 
surf bat14 pad 0.0 0.0 0.58725 0.5962500000000001 280.0 350.0 
surf clt11 pad 0.0 0.0 0.58725 0.5962500000000001 350.0 10.0 
surf clt12 pad 0.0 0.0 0.58725 0.5962500000000001 80.0 100.0 
surf clt13 pad 0.0 0.0 0.58725 0.5962500000000001 170.0 190.0 
surf clt14 pad 0.0 0.0 0.58725 0.5962500000000001 260.0 280.0 

cell bapadt11 bigT1  B4C -bat11 
cell bapadt12 bigT1  B4C -bat12 
cell bapadt13 bigT1  B4C -bat13 
cell bapadt14 bigT1  B4C -bat14 
cell clpadt11 bigT1  clad -clt11 
cell clpadt12 bigT1  clad -clt

In [40]:
CORE = sqLat("CORE1", 0, 0, 11, 21.50364)

COREMap = np.array([
[WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1],
[WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1],
[WA1, WA1, WA1, WA1, FA3, FA3, FA3, WA1, WA1, WA1, WA1],
[WA1, WA1, WA1, FA3, FA2, FA2, FA2, FA3, WA1, WA1, WA1],
[WA1, WA1, FA3, FA2, FA1, FA2, FA1, FA2, FA3, WA1, WA1],
[WA1, WA1, FA3, FA2, FA2, FA1, FA2, FA2, FA3, WA1, WA1],
[WA1, WA1, FA3, FA2, FA1, FA2, FA1, FA2, FA3, WA1, WA1],
[WA1, WA1, WA1, FA3, FA2, FA2, FA2, FA3, WA1, WA1, WA1],
[WA1, WA1, WA1, WA1, FA3, FA3, FA3, WA1, WA1, WA1, WA1],
[WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1],
[WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1]
])

CORE.setMap(COREMap)

In [41]:
width = 21.50364*8
height  = 240
radii = [0, 5, 25, 45]
mats = [clad, mod, clad]

h1 = housing(coreWidth=width, coreHeight=height, defaultCRFlag=True, radiiCR= radii, matsCR= mats)

print(h1.toString())

surf cr1 cyl 0.0 0.0 86.01456 
surf cr2 cyl 0.0 0.0 91.01456 
surf cr3 cyl 0.0 0.0 111.01456 
surf cr4 cyl 0.0 0.0 131.01456000000002 

cell cc1 housing  clad cr1 -cr2 
cell cc2 housing  water cr2 -cr3 
cell cc3 housing  clad cr3 -cr4 




In [42]:
input = core(mainUniv=CORE, housing=h1, lats=latList, pins=pinList, materials=matList)

input.setPlot(types=[1, 3], res = 10000, plotOnly = True)

In [43]:
input.writeFile("bigT.txt")

text_file = open("bigT.txt") #Display contents of created inputFile
file_content = text_file.read()
print(file_content)
text_file.close()


mat fuel1    -10.2141  burn 1  rgb 238 59 49
92235.09c	-0.04319337211
92238.09c	-0.83830401789
8016.09c	-0.11850261

mat fuel2    -10.2141  burn 1  rgb 255 165 0
92235.09c	-0.04319337211
92238.09c	-0.83830401789
8016.09c	-0.11850261

mat fuel3    -10.2141  burn 1  rgb 255 255 0
92235.09c	-0.04319337211
92238.09c	-0.83830401789
8016.09c	-0.11850261

mat water    -0.7 moder ltwr 1001   rgb 198 226 255
1001.06c	0.66667
8016.06c	0.33333

mat clad    -6.55    rgb 183 183 183
40090.06c	0.5145
40091.06c	0.1122
40092.06c	0.1715
40094.06c	0.1738
40096.06c	0.028

mat reflector    0.070462    rgb 204 153 255
40090.06c	0.0059311
40091.06c	0.0012934
40092.06c	0.001977
40094.06c	0.0020035
40096.06c	0.00032278
26054.06c	0.011217
26056.06c	0.17738
26057.06c	0.0042545
26058.06c	0.00054149
24050.06c	0.0024673
24052.06c	0.04758
24053.06c	0.0053951
24054.06c	0.001343
28058.06c	0.017122
28060.06c	0.0065978
28061.06c	0.00028421
28062.06c	0.00091398
28064.06c	0.0002329
25055.06c	0.0056687
1001.06c	0.47165
80